## imports

In [33]:
import sys
import re
#from transliterate import translit
from bs4 import BeautifulSoup, Tag
from datetime import datetime

In [34]:
import os

## functions

In [35]:
def parse_file(fpath):
    path_to_file = fpath
    with open(path_to_file) as openfile:
        file_lines = openfile.readlines()
    soup = parse_lines(file_lines) 
    augmented_soup = post_process(soup)
    tree_to_write = str(soup).replace('\n', '')
    tree_to_write = add_spaces_inline_stages(tree_to_write) #
    with open(re.sub(r'\..+', r'.xml',fpath), 'w') as outfile:
        outfile.write(tree_to_write)

In [36]:
def add_spaces_inline_stages(tree_as_string):
    tree_as_string = re.sub(r'</stage>([^\s])', r'</stage> \1', tree_as_string)
    tree_as_string = re.sub(r'([^\s])<stage>', r'\1 <stage>', tree_as_string)
    return tree_as_string

In [37]:
def add_pbstmt(filedesc):
    pubstmt_as_string = """
      <publicationStmt>
        <publisher xml:id="dracor">DraCor</publisher>
        <idno type="URL">https://dracor.org</idno>
        <availability>
          <licence>
            <ab>CC0 1.0</ab>
            <ref target="https://creativecommons.org/publicdomain/zero/1.0/">Licence</ref>
          </licence>
        </availability>
      </publicationStmt>
    """
    pbsoup = BeautifulSoup(pubstmt_as_string, 'xml')
    pbstmt = pbsoup.publicationStmt
    filedesc.append(pbstmt)

In [38]:
def add_sourcedesc(filedesc):
    sourcedesc_as_string = """
      <sourceDesc>
        <bibl type="digitalSource">
          <name>ENTER SOURCE NAME HERE</name>
          <idno type="URL">ENTER SOURCE URL HERE</idno>
          <availability status="free">
            <p>In the public domain.</p>
          </availability>
        </bibl>
      </sourceDesc>
    """
    sdsoup = BeautifulSoup(sourcedesc_as_string, 'xml')
    sd = sdsoup.sourceDesc
    filedesc.append(sd)

In [39]:
def create_header():
    header = Tag(name='teiHeader')
    fdesc = Tag(name='fileDesc')
    titlestmt = Tag(name='titleStmt')
    fdesc.append(titlestmt)
    add_pbstmt(fdesc)
    add_sourcedesc(fdesc)
    header.append(fdesc)
    return(header)

In [40]:
def get_div_level(line):
    div_level = 1 # since we already located one # and since 0 is <body> level in this model
    for char in line:
        if char == '#':
            div_level+=1
        else:
            break
    return(div_level)

In [41]:
def handle_line_with_markup(first_character, rest_of_line, current_lowest_tag, current_lowest_div, front):
    ''' handles linke with specific ezdrama markup symbol at the start
    returns new current_lowest_tag which is to be appended further'''
    if first_character == '$':
        new_stage = Tag(name='stage')
        new_stage.append(rest_of_line)
        current_lowest_div.append(new_stage)
        current_lowest_tag = new_stage # при отключениии stage перестает быть мультистрочным и поедать все немаркированные строки после себя
    elif first_character == '@':
        new_sp = Tag(name='sp')
        new_sp.append(rest_of_line)
        current_lowest_div.append(new_sp)
        current_lowest_tag = new_sp
    elif first_character == '^':
        new_cl = Tag(name='castList')
        new_cl.append(rest_of_line)
        front.append(new_cl)
        current_lowest_tag = new_cl
        
    elif first_character == '&':
        new_setting = Tag(name='set')
        new_setting.append(rest_of_line)
        front.append(new_setting)
        
    elif first_character == '#':
        new_div = Tag(name='div')
        head = Tag(name='head')
        head.append(rest_of_line.strip('#'))
        new_div['level'] = get_div_level(rest_of_line)
        new_div.append(head)
        if new_div['level'] > current_lowest_div['level']:
            current_lowest_div.append(new_div)
        elif new_div['level'] == current_lowest_div['level']:
            current_lowest_div.parent.append(new_div)
        else:
            current_lowest_div.parent.parent.append(new_div)
        current_lowest_div = new_div
        current_lowest_tag = new_div
    return(current_lowest_tag, current_lowest_div)

In [42]:
def add_author_to_header(header, line):
    fdesc = header.find('titleStmt')
    author = Tag(name='author')
    author.append(line[7:])
    fdesc.append(author)

In [43]:
def add_title_to_header(header, line):
    titlest = header.find('titleStmt')
    title = Tag(name='title')
    title['type'] = 'main'
    title.append(line[7:])
    titlest.append(title)

In [44]:
def add_subtitle_to_header(header, line):
    titlest = header.find('titleStmt')
    title = Tag(name='title')
    title['type'] = 'sub'
    title.append(line[10:])
    titlest.append(title)

In [45]:
def add_standoff(tei):
    today = datetime.today().strftime('%Y-%m-%d')
    standoff_as_string = f'''
    <standOff>
        <listEvent>
        <event type="print" when="{today}">
        <desc/>
        </event>
        <event type="premiere" when="{today}">
        <desc/>
        </event>
        <event type="written" when="{today}">
        <label>INSERT</label>
        </event>
        </listEvent>
        <listRelation>
        <relation name="wikidata" active="INSERT" passive="INSERT"/>
        </listRelation>
    </standOff>
    '''
    standoffsoup = BeautifulSoup(standoff_as_string, 'xml')
    standoff = standoffsoup.standOff
    tei.append(standoff)

In [46]:
def parse_lines(file_lines):
    '''This function parses lines of a file 
    in ezdrama format and 
    produces an output XML tree
    at this stage we only identify: 
    -- basic <div> structure
    -- external (not-inside-the-sp) <stage> directions 
    -- <sp>-s without internal markup
    '''
    special_symb_list = '@$^#&'
    root = Tag(name='TEI')
    header = create_header()
    root.append(header)
    add_standoff(root)
    text = Tag(name='text')
    front = Tag(name='front')
    body = Tag(name='body')
    text.append(front)
    text.append(body)
    root.append(text)
    
    current_lowest_tag = body
    current_lowest_div = body
    current_lowest_div['level'] = 0
    
    for line in file_lines:
        if line.startswith('@author'):
            add_author_to_header(header, line.strip())
        elif line.startswith('@title'):
            add_title_to_header(header, line.strip())
        elif line.startswith('@subtitle'):
            add_subtitle_to_header(header, line.strip())
        else:
            first_character = line[:1] # отрезаем первый спецсимвол, берем его
            rest_of_line = line[1:] # отрезаем первый спецсимвол, берем остаток
            if first_character in special_symb_list:
                current_lowest_tag, current_lowest_div = handle_line_with_markup(first_character, 
                                                                                 rest_of_line, 
                                                                                 current_lowest_tag,
                                                                                 current_lowest_div, 
                                                                                 front)
            else:
                current_lowest_tag.append(line)
    return(root)



In [47]:
def check_prose(first_line_of_speech):
    #print(first_line_of_speech)
    if first_line_of_speech.startswith('~'):
        return False
    return True

In [48]:
def clean_after_translit(line):
    line = line.replace('і', 'i')
    line = line.replace('ї', 'i')
    line = line.replace('і', 'i')
    line = line.replace('є', 'e')
    line = line.replace("'", "")
    line = line.replace("’", "")
    line = line.replace("«", "")
    line = line.replace("»", "")
    line = line.replace("′", "")
    line = line.replace(" ", "_")
    return line

In [49]:
def add_cast_items(soup):
    castList = soup.find('castList')
    casttext = castList.text
    castList.clear()
    cast_lines = casttext.split('\n')
    # first line is head
    castHead = Tag(name='head')
    castHead.append(cast_lines[0])
    castList.append(castHead)
    # next lines -- castItems
    for line in cast_lines[1:]:
        castItem = Tag(name='castItem')
        castItem.append(line)
        castList.append(castItem)

In [63]:
def post_process_sp(sp):
    text_of_sp = sp.text 
    sp.clear()
    
    text_split_in_lines = text_of_sp.split('\n')
    first_line = text_split_in_lines[0]
    speaker = Tag(name='speaker')
    sp.append(speaker)
    check_stage = re.search('([^%]+)(%%.+?%%[.,:!;]?)', first_line)
    if check_stage:
        speaker.append(check_stage.group(1))
        inside_stage = Tag(name='stage')
        inside_stage.append(check_stage.group(2).strip(' \n%'))
        sp.append(inside_stage)
    else:
        speaker.append(first_line.strip())
    
    long_stage = False
    
    for line in text_split_in_lines[1:]:
        if line.startswith('%'):
            speechtext = Tag(name='stage')
            speechtext.append(line.strip('%'))
            if line.startswith('%%'):
                long_stage = True
            else:
                sp.append(speechtext)
        elif long_stage:
            speechtext.append(f' {line.strip("%")}')
            if line.strip().endswith('%%'):
                long_stage = False
                sp.append(speechtext)
        else:
            if play_mode == 'verse':
                speechtext = Tag(name='l')
            else:
                speechtext = Tag(name='p')
            inline_stages = re.findall('(.*)(%%.+?%%[.,:!;]?)(.*)', line)
            if inline_stages:
                for triplet in inline_stages: 
                    if len(triplet[0]) > 0:
                        speechtext.append(triplet[0])
                    inside_stage = Tag(name='stage')
                    inside_stage.append(triplet[1].strip(' \n%'))
                    speechtext.append(inside_stage)
                    if len(triplet[2]) > 0:
                        speechtext.append(triplet[0])       
            else:
                speechtext.append(line)
            sp.append(speechtext)
    
    if re.search('[йцукенгшщзхъфывапролджэячсмитью]', speaker.text.lower()):
        clean_who = clean_after_translit(translit(speaker.text.strip('.: '), UkrainianISO9)).lower()
    else:
        clean_who = speaker.text.strip('.: ').lower()
    sp['who'] = f'#{clean_who}'

In [64]:
def add_rev_desc(header):
    revdesc_as_string = f"""
    <revisionDesc>
         <listChange>
        <change when="{datetime.today().strftime('%Y-%m-%d')}">DESCRIBE CHANGE</change>
        </listChange>
    </revisionDesc>"""
    rdsoup = BeautifulSoup(revdesc_as_string, 'xml')
    rd = rdsoup.revisionDesc
    header.append(rd)

In [65]:
def post_process(soup):
    set_of_char_pairs = set() # множество пар ID + строка 
    add_cast_items(soup)
    del(soup.find('body')['level'])
    for sp in soup.find_all('sp'):
        post_process_sp(sp)
        if 'who' in sp.attrs:
            #print[sp
            #try:
            set_of_char_pairs.add((sp['who'], sp.speaker.text.strip('. '))) #
            #except:
            #    pass
    for div in soup.find_all('div'):
        if div['level'] == 0:
            div.attrs = {}
        elif div['level'] == 1:
            div.attrs = {}
            div['type'] = 'act'
        elif div['level'] == 2:
            div.attrs = {}
            div['type'] = 'scene'
    add_particdesc_to_header(soup, set_of_char_pairs)
    add_rev_desc(soup.teiHeader)
    
    soup['xmlns'] = "http://www.tei-c.org/ns/1.0"
    return(soup)

In [66]:
def add_particdesc_to_header(soup, set_of_char_pairs):
    #print(set_of_char_pairs)
    profileDesc = Tag(name = 'profileDesc')
    particDesc = Tag(name = 'particDesc')
    profileDesc.append(particDesc)
    listPerson = Tag(name = 'listPerson')
    particDesc.append(listPerson)
    for pair in set_of_char_pairs:
        person = Tag(name = 'person')
        person['xml:id'] = pair[0].strip('#')
        persName = Tag(name = 'persName')
        person.append(persName)
        #print(pair[1])
        persName.append(pair[1])
        listPerson.append(person)
    teiHeader = soup.find('teiHeader')
    teiHeader.append(profileDesc)

## Use

In [67]:
#path = 'Marko_Kropivnitskij_Daj_sertsju_volju_zavede_u_nevolju.txt'
#parse_file(path)

In [68]:
#path = 'shevchenko-nazar-stodolja.md'
#parse_file(path)

In [69]:
#path = 'kvіtka-osnovjanenko-svatannja-na-goncharіvcі.md'
#parse_file(path)

In [70]:
#path = 'gogol-janovskij-prostak-abo-hitrist-zhinki-perehitrena-moskalem.md'
#parse_file(path)

In [71]:
#path = 'ezdrama_simple_demo.md'
#parse_file(path)

In [72]:
# ESP02-vega-duquesa-de-rosa

In [73]:
# path to a file in UTF-8
path = 'sample.txt'
play_mode = 'verse' #verse
parse_file(path)

In [74]:
newfile = path.replace('.txt', '.xml')

In [75]:
os.system(f'xmlformat --config-file=format.conf \"{newfile}\" > \"{newfile.replace(".xml","")}_indented.xml\" ')

/usr/local/bin/xmlformat:1459: warning: assigned but unused variable - in_file
/usr/local/bin/xmlformat:100: warning: nested repeat operator '+' and '?' was replaced with '*' in regular expression
/usr/local/bin/xmlformat:100: warning: nested repeat operator '+' and '?' was replaced with '*' in regular expression
/usr/local/bin/xmlformat:100: warning: nested repeat operator '+' and '?' was replaced with '*' in regular expression
/usr/local/bin/xmlformat:100: warning: nested repeat operator '+' and '?' was replaced with '*' in regular expression
/usr/local/bin/xmlformat:100: warning: nested repeat operator '+' and '?' was replaced with '*' in regular expression
/usr/local/bin/xmlformat:100: warning: nested repeat operator '+' and '?' was replaced with '*' in regular expression


0